In [1]:
#AutoCallable worst-off multi-assets option

In [2]:
import QuantLib as ql
import numpy as np
import scipy.optimize as opt

from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override()
import pandas as pd
from datetime import date, timedelta, datetime

np.set_printoptions(precision=9, suppress=True)

e:\!python\lib\site-packages\pandas_datareader\compat\__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [3]:
#Raytheon Technologies Corporation (RTX)
#Abiomed, Inc. (ABMD)
#Philip Morris International Inc. (PM)
#Public Joint Stock Company Mining and Metallurgical Company Norilsk Nickel (MNOD.IL)
#Twilio Inc. (TWLO)

n_stocks = 5

BASKET_TICKERS = ['RTX','ABMD','PM','MNOD.IL','TWLO']


# Fetch the data
data = pd.DataFrame(columns=BASKET_TICKERS)
dividends = pd.DataFrame(columns=['Date'])
div_yield_daily = pd.Series(index=BASKET_TICKERS,dtype='float64')

for ticker in BASKET_TICKERS:
    data[ticker] = yf.download(ticker,start=date.today() - 3*timedelta(days=365),end=date.today())['Adj Close']
    dividends = pd.merge(dividends,yf.Ticker(ticker).dividends, how='outer', on='Date').rename(columns = {'Dividends':ticker})

data.fillna(method="ffill",inplace=True)
divs_last_3y = dividends[dividends.Date>'2019-01-01'].fillna(0).set_index('Date')

for ticker in BASKET_TICKERS:
    tmp_merge = pd.merge(divs_last_3y[ticker][divs_last_3y[ticker]>0],data[ticker], how='left', on='Date').rename(
        columns = {ticker + '_x':'div',ticker + '_y':'price'})
    div_yield_daily.loc[ticker] = np.mean(tmp_merge['div']/tmp_merge['price']/365)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [4]:
def log_return(series):
    return np.log(series).diff()

def ql_to_date(d):
    return date(d.year(), d.month(), d.dayOfMonth())

def stock_price_simulation(n_stocks, S0, mue, sigma, div_yield, cov, simulationDates, simulation_rounds, seed):
    simulation = []
    dt = 1
    n = len(simulationDates)
    exp_mean = (mue - div_yield - (sigma ** 2.0) * 0.5) * dt
    exp_diffusion = sigma * np.sqrt(dt)
    
    np.random.seed(seed)
    z_t = np.random.multivariate_normal(np.zeros(n_stocks), cov, (simulation_rounds, n))
    price_array = np.array([np.zeros((n, n_stocks))] * simulation_rounds)
    
    for i in range(0, simulation_rounds):
        for j , date_step in enumerate(simulationDates):
            if j == 0:
                price_array[i, j, :] = S0
            else:
                price_array[i, j, :] = price_array[i, j - 1, :] * np.exp(exp_mean + exp_diffusion * z_t[i, j, :])
            simulation.append(np.append([ql_to_date(date_step),i,j],price_array[i, j, :]))
    return simulation

def AutoCallableBasket(valuationDate, couponDates, strike, autoCallBarrier, couponBarrier, protectionBarrier,
            hasMemory, finalRedemptionFormula, coupon, notional, dayCounter, curve,
            n_stocks, S0, mue, sigma, div_yield, cov, simulationDates, simulation_rounds, seed):    
    
    simulations_all_slices = pd.DataFrame(stock_price_simulation(n_stocks, S0, mue, sigma, div_yield, cov, 
                                                    simulationDates,simulation_rounds, seed),
                                               columns=np.append(['Date','path','time_step'], BASKET_TICKERS))
    
    simulations = simulations_all_slices[(simulations_all_slices.Date.isin(couponDates))].set_index('Date')
    
    # immediate exit trigger for matured transaction
    if(valuationDate >= np.array(couponDates)[-1]): return 0.0
    
    global_pv = []
    expirationDate = np.array(couponDates)[-1]
    hasMemory = int(hasMemory)
    
    for path in [simulations[simulations.path == i][BASKET_TICKERS] for i in range(simulation_rounds)]:
        payoffPV = 0.0
        unpaidCoupons = 0
        hasAutoCalled = False
        
        for date, index in zip(np.array(couponDates), np.min(path/strike,axis=1)):
            if(hasAutoCalled): break
            payoff = 0.0
                
            # payoff calculation at expiration
            if(date == expirationDate):
                # index is greater or equal to coupon barrier
                # pay 100% redemption, plus coupon, plus conditionally all unpaid coupons
                if(index >= couponBarrier):
                    payoff = notional * (1 + (coupon * (1 + unpaidCoupons * hasMemory)))
                # index is greater or equal to protection barrier and less than coupon barrier
                # pay 100% redemption, no coupon
                if((index >= protectionBarrier) & (index < couponBarrier)):
                    payoff = notional
                # index is less than protection barrier
                # pay redemption according to formula, no coupon
                if(index < protectionBarrier):
                    payoff = notional * finalRedemptionFormula(index)
                
            # payoff calculation before expiration
            else:
                # index is greater or equal to autocall barrier
                # autocall will happen before expiration
                # pay 100% redemption, plus coupon, plus conditionally all unpaid coupons
                if(index >= autoCallBarrier):
                    payoff = notional * (1 + (coupon * (1 + unpaidCoupons * hasMemory)))
                    hasAutoCalled = True
                # index is greater or equal to coupon barrier and less than autocall barrier
                # autocall will not happen
                # pay coupon, plus conditionally all unpaid coupons
                if((index >= couponBarrier) & (index < autoCallBarrier)):
                    payoff = notional * (coupon * (1 + unpaidCoupons * hasMemory))
                    unpaidCoupons = 0
                # index is less than coupon barrier
                # autocall will not happen
                # no coupon payment, only accumulate unpaid coupons
                if(index < couponBarrier):
                    payoff = 0.0
                    unpaidCoupons += 1                    

            if(date > valuationDate):
                df = curve.discount(ql.Date(date.day, date.month, date.year))
                payoffPV += payoff * df
            
        global_pv.append(payoffPV)
        
    return np.mean(np.array(global_pv))

In [5]:
log_return_basket = log_return(data).dropna()

mue = log_return_basket.mean().values
div_yield = div_yield_daily.fillna(0).values
sigma = log_return_basket.std().values
cov = log_return_basket.cov().values
corr = log_return_basket.corr().values
S0 = data.iloc[-1].values


print('Drift:', mue, sep='\n')
print('Dividends:', div_yield, sep='\n')
print('Volatilities:', sigma, sep='\n')
print('Covariance:', cov, sep='\n')
print('Correlation:', corr, sep='\n')
print('Initial price:', S0, sep='\n')


Drift:
[0.000332534 0.000138405 0.000611585 0.000941339 0.001623035]
Dividends:
[0.000017778 0.          0.000043352 0.000117272 0.         ]
Volatilities:
[0.023887644 0.028554851 0.018139037 0.02438539  0.03548768 ]
Covariance:
[[0.00057062  0.000158737 0.000235322 0.000250999 0.000169435]
 [0.000158737 0.00081538  0.000083571 0.000124719 0.000258639]
 [0.000235322 0.000083571 0.000329025 0.000174911 0.000086329]
 [0.000250999 0.000124719 0.000174911 0.000594647 0.000208919]
 [0.000169435 0.000258639 0.000086329 0.000208919 0.001259375]]
Correlation:
[[1.          0.232715383 0.543094715 0.430892611 0.199872079]
 [0.232715383 1.          0.161348113 0.179111313 0.255232328]
 [0.543094715 0.161348113 1.          0.395433969 0.13411063 ]
 [0.430892611 0.179111313 0.395433969 1.          0.241418847]
 [0.199872079 0.255232328 0.13411063  0.241418847 1.         ]]
Initial price:
[ 82.279998779 315.549987793  93.449996948  29.520000458 273.630004883]


In [6]:
# general QuantLib-related parameters
ql.Settings.instance().evaluationDate = ql.Date.todaysDate()
convention = ql.ModifiedFollowing
dayCounter = ql.Actual360()
calendar = ql.UnitedStates(ql.UnitedStates.NYSE)
curve = ql.YieldTermStructureHandle(ql.FlatForward(ql.Date.todaysDate(), 0.04, dayCounter))

# initial parameters
spot = S0
strike = S0
autoCallBarrier = 1.0
couponBarrier = 0.65
protectionBarrier = 0.55
finalRedemptionFormula = lambda indexAtMaturity: min(1.0, indexAtMaturity)
coupon = 0.0425
notional = 1250
hasMemory = True

simulation_rounds = 1000
seed = 777

# coupon schedule for basket
startDate = ql.Date.todaysDate()
firstCouponDate = calendar.advance(startDate, ql.Period('3m'))
lastCouponDate = calendar.advance(startDate, ql.Period(5, ql.Years))
couponDates = pd.Series(list(ql.Schedule(firstCouponDate, lastCouponDate, ql.Period(ql.Quarterly), 
    calendar, ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False))).apply(ql_to_date)

#simulation dates
simulationDates = np.array(list(ql.Schedule(startDate, lastCouponDate, ql.Period('1d'), 
    ql.NullCalendar(), ql.ModifiedFollowing, ql.ModifiedFollowing, ql.DateGeneration.Forward, False)))
valuationDate = date.today()

In [7]:
PV = AutoCallableBasket(valuationDate, couponDates, strike, autoCallBarrier, couponBarrier, protectionBarrier,
    hasMemory, finalRedemptionFormula, coupon, notional, dayCounter, curve,
    n_stocks, S0, mue, sigma, div_yield, cov, simulationDates, simulation_rounds, seed)

In [8]:
print('Present Value of AutoCallableBasket (USD):', round(PV,2))

Present Value of AutoCallableBasket (USD): 1845.96
